# Generate training and validation data for OpenAI Fine-tuning
This notebook provides a process for creating training and validation data for the OpenAI Fine-tuning API from a CSV file.
Training and validation data is saved in JSONL format where each line is a JSON object of a completed exchange between the API user and assistant. 
Example of completed exchange:
```JSON
{
    'messages': [
        {
            'role': 'system',
            'content': 'You are a restaurant review analyzer.'
        },
        {
            'role': 'user',
            'content': 'Review: The food was wonderful!'
        },
        {
            'role': 'assistant',
            'content': '{"rating": 1, "sentiment": "Culinary Catastrophe"}'
        }
    ]
}
```
## Setup
1. __Load the data__: Point to the original CSV data file.
2. __Create example exchanges__: Configure the example exchanges to match your data and output needs.
3. __Save the data__: Set the desired filenames for the training and testing data.
4. __Validate data__: Enter the filename you want to validate and run the function.
5. __Get token counts__: Run the function to get the token counts and warnings for the specified file.
6. __Cost estimation__: Run the function to get the total token count across all epochs for cost estimation of the specified file.
## Sources
This example borrows liberally from ["How to fine-tune chat models](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb) by OpenAI.
The example data file `data/allreviews.csv` was created using parts of the ["10000 Restaurant Reviews"](https://www.kaggle.com/datasets/joebeachcapital/restaurant-reviews) dataset from Kaggle.

In [ ]:
# Import necessary libraries
import os
import pandas as pd
import json         
from pprint import pprint
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

## 1. Load the data

In [ ]:
# Data setup
data_folder = "../data"
source_file = os.path.join(data_folder, "allreviews.csv")
training_file_name = os.path.join(data_folder, "review_finetune_training.jsonl")
validation_file_name = os.path.join(data_folder, "review_finetune_validation.jsonl")

# Load the data
data = pd.read_csv(os.path.join(data_folder, source_file))
data.head()

## 2. Create example exchanges

In [ ]:
# training_data = []

# The system message is the same for all examples.
system_message = "You are a restaurant review analyzer. Analyze the sentiment in the review provided. Output the rating value and the sentiment as JSON ."

# Create the user message.
def create_user_message(row):
    return f"""Review: {row['Review']}"""

# Create the assistant message.
def create_assistant_message(row):
    return f"{{\"rating\": {row['Rating']}, \"Sentiment\": \"{row['Sentiment']}\"}}\n"

# Receive the current row and create an example exchange.
def create_full_exchange(row):
    messages = []
    messages.append({"role": "system", "content": system_message})

    user_message = create_user_message(row)
    messages.append({"role": "user", "content": user_message})

    assistant_message = create_assistant_message(row)
    messages.append({"role": "assistant", "content": assistant_message})

    return {"messages": messages}

# Output the first example exchange.
pprint(create_full_exchange(data.iloc[0]))

In [ ]:
# Pull the first 100 rows of data for the training set and the next 100 for the validation set.
training_df = data.iloc[0:100]
validation_df = data.iloc[100:200]

# Use create_full_exchange on each row to generate the example exchanges.
training_data = training_df.apply(create_full_exchange, axis=1).tolist()
validation_data = validation_df.apply(create_full_exchange, axis=1).tolist()

In [ ]:
# Output the first five examples in the training set.
for example in training_data[:5]:
    print(example)

## 3. Create JSONL files

In [ ]:
# Save the data as .jsonl files. Each line is a full example exchange.
# NOTE: In JSONL format, each line ends with a line break without a comma.
def write_jsonl(data_list: list, filename: str) -> None:
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [ ]:
# Write to .jsonl files
write_jsonl(training_data, training_file_name)
write_jsonl(validation_data, validation_file_name)

## 4. Validate the data

In [ ]:
# Point to a data file to test (one at a time)
file_to_test = training_file_name
# file_to_test = validation_file_name

# Load the dataset
with open(file_to_test, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

In [ ]:
from check_format_errors import check_format_errors
# Check the format of the dataset
errors = check_format_errors(dataset)

# 5. Get token counts

In [ ]:
from token_counter import num_tokens_from_messages, num_assistant_tokens_from_messages, print_distribution

# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

# 6. Cost estimation

In [ ]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")